# pymysql
- 파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
- Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따른다.
- https://github.com/PyMySQL/PyMySQL/
- https://pymysql.readthedocs.io/en/latest/

# 설치
- 조건
    - python version 3.6 이상
    - mysql version 5.6 이상
- 설치
    - `pip install PyMySQL`
    - `conda install -c conda-forge pymysql`

In [5]:
!python -m pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 22.1
    Uninstalling pip-22.1:
      Successfully uninstalled pip-22.1


In [6]:
!pip install PyMySQL

# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host="DBMS 서버 ip", 
                                     port="port번호", 
                                     user="계정명", 
                                     password="비밀번호", 
                                     db="연결할데이터베이스이름", 
                                     charset='utf8')
    ```
    - port 번호 기본값: 3306
2. Connection을 이용해 Cursor 생성
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    ```python
        cursor = connection.cursor()
    ```
3. Cusror를 이용해 SQL문 전송
    ```python
        cursor.execute("sql문")
    ```
4. 연결 닫기
    - cursor, connection 연결을 닫는다.
    - with문을 이용할 수 있다. 
    ```python
    cursor.close()
    connection.close()
    ```

# 예제

## 테이블 생성
- 참고) pymysql을 이용해서는 DML(insert, update, delete, update)문을 실행한다. (DDL은 거의 실행하지 않는다.)

In [7]:
# import
import pymysql

# DB 연결 - connect()
connection = pymysql.connect(host = '127.0.0.1',
                             port = 3306,
                             user = 'scott',
                             password = 'tiger',
                             db = 'testdb')
print(type(connection))

<class 'pymysql.connections.Connection'>


In [8]:
# Cursor를 Connection으로 부터 얻어온다 - Cursor: sql문 실행을 관리하는 객체.
cursor = connection.cursor()
print(type(cursor))

<class 'pymysql.cursors.Cursor'>


In [12]:
# sql 문 실행
# sql ansdms string을 정의. execute() 메소드를 이용해서 DB에 전송
sql = """
create table test_user(
 id int auto_increment primary key,
 name varchar(30) not null,
 email varchar(100) not null unique,
 tall decimal(5,2),
 birthday date
)
"""
cursor.execute(sql)  # 실행

0

In [13]:
# 연결 닫기 - cursor, connection close
cursor.close()
connection.close()

## DML
### insert

In [34]:
import pymysql # auto commit = False
sql = "insert into test_user(name, email, tall, birthday) values ('홍길동', 'hong@a.com', 182.23, '2000-02-03')"
try:
    connection = pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb',charset='utf8')
    # cursor 생성
    cursor = connection.cursor()
    # insert문 실행
    cnt = cursor.execute(sql)
    print(f"{cnt}행이 insert되었습니다.")
    connection.commit() # insert/delete/update 문 실행 후 commit 처리를 해야한다
except Exception as e:
    if connection:
        connection.rollback()
    print(e)
finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()

1행이 insert되었습니다.


In [22]:
# with 문
sql = "insert into test_user(name, email, tall, birthday) values ('홍길동2', 'hong2@a.com', 182.23, '2000-02-03')"
with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
    with connection.cursor() as cursor:
        cnt = cursor.execute(sql)
        print(f'{cnt}행이 insert됨')
        connection.commit()

1행이 insert됨


In [23]:
def insert_user(name, email, tall, birthday):
    sql = f"insert into test_user(name, email, tall, birthday) values ('{name}', '{email}', {tall}, '{birthday}')"
    with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
        with connection.cursor() as cursor:
            cnt = cursor.execute(sql)
            print(f'{cnt}행이 insert됨')
            connection.commit()
    

In [24]:
insert_user('이순신', 'lee@a.com', 190.20, '1995-10-22')

1행이 insert됨


### Parameterized Query
- Parameterized Query
    - SQL 문에서 컬럼 값이 들어가는 자리에 값대신 `%s` placeholder를 사용한뒤 execute()에서 placeholder에 넣을 값을 tuple로 제공한다.
    - query문을 쉽게 작성할 수 있는 장점이 있다.        

In [27]:
date(1999,3,2) # sql date

datetime.date(1999, 3, 2)

In [31]:
# datatime 모듈: 날짜-date, 시간-time, 날짜시간-datetime 등 날짜와 시간을 다루는 모듈. => DB의 날짜/시간 타입과 연동.
from datetime import date
sql = "insert into test_user(name, email, tall, birthday) values (%s, %s, %s, %s)"
with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
    with connection.cursor() as cursor:
        cnt = cursor.execute(sql, ('유관순', 'yoo2@a.com', 175.3, '2000-01-05'))
        cnt2 = cursor.execute(sql, ('강감찬', 'kang@a.com', 170.3, date(1995,2,3)))
        print(f'{cnt}행이 insert됨')
        print(f'{cnt2}행이 insert됨')
        connection.commit()

1행이 insert됨
1행이 insert됨


### Parameterized Query를 이용해 여러개 행 insert

#### for문 사용

In [37]:
names = ['이름1', '이름2', '이름3']
emails = ['e1@a.com','e2@a.com','e3@a.com']
talls = [182.3, 157.3, 175.2]
birthdays = ['2012.2.3',date.today(), '1952-5-3']

In [53]:
sql = "insert into test_user(name, email, tall, birthday) values (%s, %s, %s, %s)"
with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
    with connection.cursor() as cursor:
        for data in zip(names, emails, talls,birthdays):
            cursor.execute(sql,data)
        connection.commit()
        
        

IntegrityError: (1062, "Duplicate entry 'e1@a.com' for key 'test_user.email'")

#### executemany() 사용
- insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.

In [85]:
datas = [
    ['name1', 'a1@a.com', 172.3, '2002-02-22'],
    ['name2', 'a2@a.com', 145.23, '2001-03-20'],
    ['name3', 'a3@a.com', 178.2, '2015-04-23']
]
# datas = [
#     ['name1', 'a1@a.com', 172.3, '2000-02-22'],
#     ['name2', 'a2@a.com', 182.44, '2001-03-20'],
#     ['name3', 'a3@a.com', 162.44, '2005-03-10'],
# ]
sql = 'insert into test_user(name, email, tall, birthday) values (%s, %s, %s, %s)'
with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
    with connection.cursor() as cursor:
        cursor.executemany(sql, datas)
        connection.commit()
        

IntegrityError: (1062, "Duplicate entry 'a1@a.com' for key 'test_user.email'")

In [75]:
datas = [
    ['name1', 'a1@a.com', 172.3, '2000-02-22'],
    ['name2', 'a2@a.com', 182.44, '2001-03-20'],
    ['name3', 'a3@a.com', 162.44, '2005-03-10'],
]
sql = 'insert into test_user (name, email, tall, birthday) values (%s, %s, %s, %s)'
with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
    with connection.cursor() as cursor:
        cursor.executemany(sql, datas)
        connection.commit()
        

IntegrityError: (1062, "Duplicate entry 'a1@a.com' for key 'test_user.email'")

In [57]:
update_sql = "update test_user set tall = tall+10, birthday = '2000-01-01'"
with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
    with connection.cursor() as cursor:
        cnt = cursor.execute(update_sql)
        connection.commit()
        print(f'{cnt}행이 update됨')
        

9행이 update됨


### update/delete
- 코딩 절차는 insert 와 동일

In [82]:
def update_user_by_id(id, name, email, tall, birthday):
    update_sql = "update test_user set name=%s, email=%s, tall=%s, birthday=%s where id=%s"
    with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
        with connection.cursor() as cursor:
            cnt = cursor.execute(update_sql, (name, email, tall, birthday, id))
            connection.commit()
            return cnt
        
        
        

In [83]:
update_user_by_id(2, '홍길동', 'honggildong@abc.com', 192.23, '1990-10-10')

0

In [ ]:
def update_user_by_id(id, name, email, tall, birthday):
    update_sql = 'update test_user set name=%s, email=%s, tall=%s, birthday=%s where id=%s'
    with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
        with connection.cursor() as cursor:
            cnt = cursor.execute(update_sql, (name, email, tall, birthday, id))
            connection.commit()
            return cnt

In [ ]:
update_user_by_id(2, '홍길동', 'honggildong@abc.com', 192.23, '1990-10-10')

In [60]:
# delete
delete_sql = "delete from test_user where id =%s"
with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
        with connection.cursor() as cursor:
            cnt = cursor.execute(delete_sql,(3,))
            print(cnt)
            connection.commit()

1


In [61]:
def delete_user_by_id(id):
    delete_sql = "delete from test_user where id =%s"
    with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
            with connection.cursor() as cursor:
                cnt = cursor.execute(delete_sql,(id,))
                return cnt

In [63]:
delete_user_by_id(4)

1


1

## select (DQL - Data Query Language)
- 조회결과 조회
    - cursor.execute("select문") 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받는다.
- fetch메소드
    - **fetchall()**
        - 조회한 모든 행을을 반환
    - **fetchmany(size=개수)**
        - 지정한 size개수 만큼 반환
    - **fetchone()**
        - 조회결과 중 첫번째 행만 반환
        - 주로 pk 동등 조건으로 조회한 경우 사용

### fetchall()

In [12]:
import pymysql
sql = "select * from test_user"

with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb',charset='utf8') as connection:
    with connection.cursor() as cursor:
#        select문 실행
        print(type(cursor))
        cursor.execute(sql)
#       select 결과 조회 (cursor.fetchxxx()메소드 이용)
        result=cursor.fetchall()

<class 'pymysql.cursors.Cursor'>


In [9]:
print(type(result),len(result))
result

<class 'tuple'> 11


((4, '이순신', 'lee@a.com', Decimal('200.20'), datetime.date(2000, 1, 1)),
 (5, '유관순', 'yoo@a.com', Decimal('185.30'), datetime.date(2000, 1, 1)),
 (10, '유관순', 'yoo2@a.com', Decimal('185.30'), datetime.date(2000, 1, 1)),
 (11, '강감찬', 'kang@a.com', Decimal('180.30'), datetime.date(2000, 1, 1)),
 (14, '홍길동', 'hong@a.com', Decimal('192.23'), datetime.date(2000, 1, 1)),
 (18, '이름1', 'e1@a.com', Decimal('192.30'), datetime.date(2000, 1, 1)),
 (19, '이름2', 'e2@a.com', Decimal('167.30'), datetime.date(2000, 1, 1)),
 (20, '이름3', 'e3@a.com', Decimal('185.20'), datetime.date(2000, 1, 1)),
 (22, 'name1', 'a1@a.com', Decimal('172.30'), datetime.date(2000, 2, 22)),
 (23, 'name2', 'a2@a.com', Decimal('182.44'), datetime.date(2001, 3, 20)),
 (24, 'name3', 'a3@a.com', Decimal('162.44'), datetime.date(2005, 3, 10)))

In [13]:
sql = "select * from test_user"

with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger',db='testdb',charset='utf8',
                    cursorclass=pymysql.cursors.DictCursor) as connection: # 조회결과를 Dictionary로 반환하는 커서
    with connection.cursor() as cursor:
        print(type(cursor))
        cursor.execute(sql)
        result = cursor.fetchall()
print("2번행의 이름",result[1][1])

<class 'pymysql.cursors.DictCursor'>


In [14]:
print(type(result)),len(result)
print(type(result[0])) # dict: key -컬럼명, value- 컬럼값
result

<class 'list'>
<class 'dict'>


[{'id': 4,
  'name': '이순신',
  'email': 'lee@a.com',
  'tall': Decimal('200.20'),
  'birthday': datetime.date(2000, 1, 1)},
 {'id': 5,
  'name': '유관순',
  'email': 'yoo@a.com',
  'tall': Decimal('185.30'),
  'birthday': datetime.date(2000, 1, 1)},
 {'id': 10,
  'name': '유관순',
  'email': 'yoo2@a.com',
  'tall': Decimal('185.30'),
  'birthday': datetime.date(2000, 1, 1)},
 {'id': 11,
  'name': '강감찬',
  'email': 'kang@a.com',
  'tall': Decimal('180.30'),
  'birthday': datetime.date(2000, 1, 1)},
 {'id': 14,
  'name': '홍길동',
  'email': 'hong@a.com',
  'tall': Decimal('192.23'),
  'birthday': datetime.date(2000, 1, 1)},
 {'id': 18,
  'name': '이름1',
  'email': 'e1@a.com',
  'tall': Decimal('192.30'),
  'birthday': datetime.date(2000, 1, 1)},
 {'id': 19,
  'name': '이름2',
  'email': 'e2@a.com',
  'tall': Decimal('167.30'),
  'birthday': datetime.date(2000, 1, 1)},
 {'id': 20,
  'name': '이름3',
  'email': 'e3@a.com',
  'tall': Decimal('185.20'),
  'birthday': datetime.date(2000, 1, 1)},
 {'id': 22

In [19]:
sql = "select * from test_user"

with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger',db='testdb',charset='utf8',
                    ) as connection: # 조회결과를 Dictionary로 반환하는 커서
    with connection.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
print(result)
print("2번행의 이름",result[1][1])

((4, '이순신', 'lee@a.com', Decimal('200.20'), datetime.date(2000, 1, 1)), (5, '유관순', 'yoo@a.com', Decimal('185.30'), datetime.date(2000, 1, 1)), (10, '유관순', 'yoo2@a.com', Decimal('185.30'), datetime.date(2000, 1, 1)), (11, '강감찬', 'kang@a.com', Decimal('180.30'), datetime.date(2000, 1, 1)), (14, '홍길동', 'hong@a.com', Decimal('192.23'), datetime.date(2000, 1, 1)), (18, '이름1', 'e1@a.com', Decimal('192.30'), datetime.date(2000, 1, 1)), (19, '이름2', 'e2@a.com', Decimal('167.30'), datetime.date(2000, 1, 1)), (20, '이름3', 'e3@a.com', Decimal('185.20'), datetime.date(2000, 1, 1)), (22, 'name1', 'a1@a.com', Decimal('172.30'), datetime.date(2000, 2, 22)), (23, 'name2', 'a2@a.com', Decimal('182.44'), datetime.date(2001, 3, 20)), (24, 'name3', 'a3@a.com', Decimal('162.44'), datetime.date(2005, 3, 10)))
2번행의 이름 유관순


In [17]:
result[0]['name'], result[0]['email']

('이순신', 'lee@a.com')

### fetchone()

In [15]:
sql = "select * from test_user where id = %s"
# pk로 조회 - 조회결과 0 또는 1행
with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb',charset='utf8') as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql,(4))  
        result=cursor.fetchone() # 조회결과가 없으면 None 있으면 1개의 데이터 반환 => where에서 pk로 조회할 경우
print(result)

(4, '이순신', 'lee@a.com', Decimal('200.20'), datetime.date(2000, 1, 1))


In [8]:
sql = "select * from test_user"
# pk로 조회 - 조회결과 0 또는 1행
with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb',charset='utf8') as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql) 
        result=cursor.fetchone() # 여러행이 조회될경우 한행만(첫행만) 반환

### fetchmany()

In [31]:
sql = "select * from test_user"
with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb',charset='utf8') as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchmany(size = 3)
        
        
        

In [28]:
print(len(result))
print(result)

11
((4, '이순신', 'lee@a.com', Decimal('200.20'), datetime.date(2000, 1, 1)), (5, '유관순', 'yoo@a.com', Decimal('185.30'), datetime.date(2000, 1, 1)), (10, '유관순', 'yoo2@a.com', Decimal('185.30'), datetime.date(2000, 1, 1)), (11, '강감찬', 'kang@a.com', Decimal('180.30'), datetime.date(2000, 1, 1)), (14, '홍길동', 'hong@a.com', Decimal('192.23'), datetime.date(2000, 1, 1)), (18, '이름1', 'e1@a.com', Decimal('192.30'), datetime.date(2000, 1, 1)), (19, '이름2', 'e2@a.com', Decimal('167.30'), datetime.date(2000, 1, 1)), (20, '이름3', 'e3@a.com', Decimal('185.20'), datetime.date(2000, 1, 1)), (22, 'name1', 'a1@a.com', Decimal('172.30'), datetime.date(2000, 2, 22)), (23, 'name2', 'a2@a.com', Decimal('182.44'), datetime.date(2001, 3, 20)), (24, 'name3', 'a3@a.com', Decimal('162.44'), datetime.date(2005, 3, 10)))


In [32]:
print(len(result))
print(result)

3
((4, '이순신', 'lee@a.com', Decimal('200.20'), datetime.date(2000, 1, 1)), (5, '유관순', 'yoo@a.com', Decimal('185.30'), datetime.date(2000, 1, 1)), (10, '유관순', 'yoo2@a.com', Decimal('185.30'), datetime.date(2000, 1, 1)))


### cursor 는 iterable 타입
- for문에 select 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.

In [33]:
sql = "select * from test_user"
with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb',charset='utf8') as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        for data in cursor: # 한번 반복시마다 조회결과를 한행씩 반환
            print(data)
            

(4, '이순신', 'lee@a.com', Decimal('200.20'), datetime.date(2000, 1, 1))
(5, '유관순', 'yoo@a.com', Decimal('185.30'), datetime.date(2000, 1, 1))
(10, '유관순', 'yoo2@a.com', Decimal('185.30'), datetime.date(2000, 1, 1))
(11, '강감찬', 'kang@a.com', Decimal('180.30'), datetime.date(2000, 1, 1))
(14, '홍길동', 'hong@a.com', Decimal('192.23'), datetime.date(2000, 1, 1))
(18, '이름1', 'e1@a.com', Decimal('192.30'), datetime.date(2000, 1, 1))
(19, '이름2', 'e2@a.com', Decimal('167.30'), datetime.date(2000, 1, 1))
(20, '이름3', 'e3@a.com', Decimal('185.20'), datetime.date(2000, 1, 1))
(22, 'name1', 'a1@a.com', Decimal('172.30'), datetime.date(2000, 2, 22))
(23, 'name2', 'a2@a.com', Decimal('182.44'), datetime.date(2001, 3, 20))
(24, 'name3', 'a3@a.com', Decimal('162.44'), datetime.date(2005, 3, 10))


In [ ]:
def insert_user(name, email, tall, birthday):
    sql = f"insert into test_user(name, email, tall, birthday) values ('{name}', '{email}', {tall}, '{birthday}')"
    with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
        with connection.cursor() as cursor:
            cnt = cursor.execute(sql)
            print(f'{cnt}행이 insert됨')
            connection.commit()

## TODO
- DB:hr (emp 테이블)

In [96]:
# 1개의 행을 emp 테이블에 insert 하는 함수 - 호출해서 값 insert
# Null을 허용하는 컬럼, default값이 있는 컬럼들은 기본값이 있는 매개변수로 선언한다. (null=>None)
def insert_emp(emp_id, emp_name, job, hire_date, mgr_id=None, salary=None, comm_pct=None, dept_name=None):
    sql = 'insert into emp values(%s, %s, %s, %s, %s, %s, %s, %s)'
    with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='hr', charset='utf8') as connection:
        with connection.cursor() as cursor:
            cnt = cursor.execute(sql, (emp_id, emp_name, job, mgr_id, hire_date, salary, comm_pct, dept_name))
            print(type(cnt))
            connection.commit()
            return cnt

In [97]:
insert_emp(403, '홍길동', '기획', '2020-10-10', 100, 3000, 0.2, '기획부')

<class 'int'>


1

In [ ]:
# emp 테이블의 데이터를 수정하는 함수 - emp_id(where)로 나머지 정보를 수정

In [62]:
def update_emp(emp_id, emp_name, job, salary, mgr_id, hire_date, comm_pct, dept_name):
    sql = "update emp set emp_name=%s, job=%s, salary=%s, mgr_id=%s, hire_date=%s, comm_pct=%s, dept_name=%s where emp_id = %s"
    with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='hr', charset='utf8') as connection:
        with connection.cursor() as cursor:
            cnt = cursor.execute(sql, (emp_name, job, salary, mgr_id, hire_date, comm_pct, dept_name, emp_id))
            connection.commit()
            return cnt
            

In [63]:
# update_emp(206, '윌리암', '회계', '2002-06-07', 205,  18300, None, 'Accounting')
update_emp(206, '월리암', '회계', 18300, 205, '2002-06-07', None, 'Accounting')

1

In [ ]:
# emp_id를 매개변수로 받아서 그 emp_id의 직원 정보를 삭제하는 메소드

In [70]:
def delete_emp(emp_id):
    sql = "delete from emp where emp_id = %s"
    with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='hr', charset='utf8') as connection:
        with connection.cursor() as cursor:
            cnt = cursor.execute(sql,(emp_id))
            connection.commit()
            return cnt

In [71]:
delete_emp(205)

1

In [ ]:
# job을 매개변수로 받아서 그 job의 직원들을 삭제하는 메소드

In [74]:
def delete_job(job):
    sql = "delete from emp where job = %s"
    with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='hr', charset='utf8') as connection:
        with connection.cursor() as cursor:
            cnt = cursor.execute(sql,(job))
            connection.commit()
            return cnt

In [75]:
delete_job('PR_REP')

1

In [ ]:
# 전체 직원을 조회한 결과를 반환하는 메소드.

In [87]:
def select_all_emp():
    sql = "select * from emp"
    with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='hr', charset='utf8') as connection:
        with connection.cursor() as cursor:
            cursor.execute(sql)
            result = cursor.fetchall()
            return result

In [89]:
r= select_all_emp()
print(len(r))
print(r)

109
((100, 'Steven', 'AD_PRES', None, datetime.date(2003, 6, 17), Decimal('24000.00'), None, 'Executive'), (101, 'Neena', 'AD_VP', 100, datetime.date(2005, 9, 21), Decimal('17000.00'), None, 'Executive'), (102, 'Lex', 'AD_VP', 100, datetime.date(2001, 1, 13), Decimal('17000.00'), None, 'Executive'), (103, 'Alexander', 'IT_PROG', 102, datetime.date(2006, 1, 3), Decimal('9000.00'), None, 'IT'), (104, 'Bruce', 'IT_PROG', 103, datetime.date(2007, 5, 21), Decimal('6000.00'), None, 'IT'), (105, 'David', 'IT_PROG', 103, datetime.date(2005, 6, 25), Decimal('4800.00'), None, 'IT'), (106, 'Valli', 'IT_PROG', 103, datetime.date(2006, 2, 5), Decimal('4800.00'), None, 'IT'), (107, 'Diana', 'IT_PROG', 103, datetime.date(2007, 2, 7), Decimal('4200.00'), None, 'IT'), (108, 'Nancy', 'FI_MGR', 101, datetime.date(2002, 8, 17), Decimal('12008.00'), None, 'Finance'), (109, 'Daniel', 'FI_ACCOUNT', 108, datetime.date(2002, 8, 16), Decimal('9000.00'), None, 'Finance'), (110, 'John', 'FI_ACCOUNT', 108, datetim

In [ ]:
# emp_id 를 매개변수로 받아서 그 직원 정보를 조회한 결과를 반환하는 메소드

In [5]:
import pymysql
def select_emp_id_emp(emp_id):
    sql = "select * from emp where emp_id = %s"
    with pymysql.connect(host='127.0.0.1',port=3306,user='scott',password='tiger',db='hr',charset='utf8') as connection:
        with connection.cursor() as cursor:
            cursor.execute(sql,(emp_id))
            result = cursor.fetchall()
            return result

        

In [6]:
select_emp_id_emp(401)

((401,
  '홍길동',
  '기획',
  100,
  datetime.date(2020, 10, 10),
  Decimal('3000.00'),
  Decimal('0.20'),
  '기획부'),)

In [ ]:
# dept_name을 매개변수로 받아서 그 부서의 직원들을 조회한 결과를 반환하는 메소드

In [105]:
def select_dept_name(dept_name):
    sql = "select * from emp where dept_name = %s"
    with pymysql.connect(host='127.0.0.1',port=3306,user='scott',password='tiger',db='hr',charset='utf8') as connection:
        with connection.cursor() as cursor:
            cursor.execute(sql,(dept_name))
            result = cursor.fetchall()
            return result
    

In [106]:
select_dept_name('Shipping')

((120,
  'Matthew',
  'ST_MAN',
  100,
  datetime.date(2004, 7, 18),
  Decimal('8000.00'),
  None,
  'Shipping'),
 (121,
  'Adam',
  'ST_MAN',
  100,
  datetime.date(2005, 4, 10),
  Decimal('8200.00'),
  None,
  'Shipping'),
 (122,
  'Payam',
  'ST_MAN',
  100,
  datetime.date(2003, 5, 1),
  Decimal('7900.00'),
  None,
  'Shipping'),
 (123,
  'Shanta',
  'ST_MAN',
  100,
  datetime.date(2005, 10, 10),
  Decimal('6500.00'),
  None,
  'Shipping'),
 (124,
  'Kevin',
  'ST_MAN',
  100,
  datetime.date(2007, 11, 16),
  Decimal('5800.00'),
  None,
  'Shipping'),
 (125,
  'Julia',
  'ST_CLERK',
  120,
  datetime.date(2005, 7, 16),
  Decimal('3200.00'),
  None,
  'Shipping'),
 (126,
  'Irene',
  'ST_CLERK',
  120,
  datetime.date(2006, 9, 28),
  Decimal('2700.00'),
  None,
  'Shipping'),
 (127,
  'James',
  'ST_CLERK',
  120,
  datetime.date(2007, 1, 14),
  Decimal('2400.00'),
  None,
  'Shipping'),
 (128,
  'Steven',
  'ST_CLERK',
  120,
  datetime.date(2008, 3, 8),
  Decimal('2200.00'),
  No

In [ ]:
# salary 범위를 받아서 그 범위의 salary를 받는 직원들을 조회한 결과를 반환하는 함수

In [108]:
def select_salary(min_salary, max_salary):
    sql = "select * from emp where salary between %s and %s"
    with pymysql.connect(host='127.0.0.1',port=3306,user='scott',password='tiger',db='hr',charset='utf8') as connection:
        with connection.cursor() as cursor:
            cursor.execute(sql,(min_salary,max_salary))
            result = cursor.fetchall()
            return result
            

In [109]:
select_salary(3000, 4000)

((125,
  'Julia',
  'ST_CLERK',
  120,
  datetime.date(2005, 7, 16),
  Decimal('3200.00'),
  None,
  'Shipping'),
 (129,
  'Laura',
  'ST_CLERK',
  121,
  datetime.date(2005, 8, 20),
  Decimal('3300.00'),
  None,
  'Shipping'),
 (133,
  'Jason',
  'ST_CLERK',
  122,
  datetime.date(2004, 6, 14),
  Decimal('3300.00'),
  None,
  'Shipping'),
 (137,
  'Renske',
  'ST_CLERK',
  123,
  datetime.date(2003, 7, 14),
  Decimal('3600.00'),
  None,
  'Shipping'),
 (138,
  'Stephen',
  'ST_CLERK',
  123,
  datetime.date(2005, 10, 26),
  Decimal('3200.00'),
  None,
  'Shipping'),
 (141,
  'Trenna',
  'ST_CLERK',
  124,
  datetime.date(2003, 10, 17),
  Decimal('3500.00'),
  None,
  'Shipping'),
 (142,
  'Curtis',
  'ST_CLERK',
  124,
  datetime.date(2005, 1, 29),
  Decimal('3100.00'),
  None,
  'Shipping'),
 (180,
  'Winston',
  'SH_CLERK',
  120,
  datetime.date(2006, 1, 24),
  Decimal('3200.00'),
  None,
  'Shipping'),
 (181,
  'Jean',
  'SH_CLERK',
  120,
  datetime.date(2006, 2, 23),
  Decimal('

In [ ]:
# dept_name을 매개변수로 받아서 부서별 급여 통계정보(합계, 평균, 최대, 최소)를 반환하는 함수

In [118]:
def select_deft_name_salary(dept_name):
    sql = "select max(salary) 최대, min(salary) 최소, avg(salary) 평균, sum(salary) 합계 from emp where dept_name = %s"
    with pymysql.connect(host='127.0.0.1',port=3306,user='scott',password='tiger',db='hr',charset='utf8',
                        cursorclass=pymysql.cursors.DictCursor) as connection:
        with connection.cursor() as cursor:
            cursor.execute(sql,(dept_name))
            result = cursor.fetchall()
            return result
            

In [119]:
select_deft_name_salary('IT')

[{'최대': Decimal('9000.00'),
  '최소': Decimal('4200.00'),
  '평균': Decimal('5760.000000'),
  '합계': Decimal('28800.00')}]